In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import os
import sys
from utils import fuzzy_merge

from ast import literal_eval

try:
    from cfuzzyset import cFuzzySet as FuzzySet
except ImportError:
    from fuzzyset import FuzzySet

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
pt_df = pd.read_csv("./data/pt_ratings.csv")
salaries_df = pd.read_csv("./data/salaries.csv")
rmp_df = pd.read_csv("./data/rmp_ratings.csv")

In [3]:
# Join names to help name search in salaries
salaries_df["name"] = salaries_df["employee"].apply(
    lambda x: " ".join(x.split(", ")[::-1])
)

# pt_df = pt_df[pt_df["reviews"] == "[]"].drop)
# rmp_df = rmp_df.dropna(subset=["reviews"])

In [4]:
pt_df

,courses,average_rating,type,reviews,name,slug
0,"['INST737', 'ENPM808W', 'ENPM808W', 'ENPM808W']",NaN,professor,[],A Seyed,seyed
1,"['MUSC800W', 'MUSC830W', 'MUSC830W']",NaN,professor,[],Aaron Goldman,goldman_aaron
2,"['THET678', 'THET499']",NaN,professor,[],Aaron Posner,posner
3,"['NFSC431', 'NFSC679R', 'NFSC112', 'HLTH672', ...",NaN,professor,[],Abani Pradhan,pradhan
4,"['CMLT235', 'ENGL101', 'ENGL234', 'ENGL101H', ...",NaN,professor,[],Abbey Morgan,morgan_abbey
...,...,...,...,...,...,...
11835,[],2.0,professor,"[{'professor': 'Lance Shapiro', 'course': 'BSC...",Lance Shapiro,shapiro_lance
11836,[],5.0,ta,"[{'professor': 'Livingstone Imonitie', 'course...",Livingstone Imonitie,imonitie
11837,[],5.0,ta,"[{'professor': 'Kathryn Lawless', 'course': 'U...",Kathryn Lawless,lawless_kathryn
11838,[],5.0,ta,"[{'professor': 'Matt Finnan', 'course': 'ENEE1...",Matt Finnan,finnan_matt


In [5]:
merge_pt = fuzzy_merge(pt_df, salaries_df, fuzz_on="name", how="inner")
merge_pt


,courses,average_rating,type,reviews,name_x,slug,name,year,employee,department,division,title,salary,name_y
0,"['MUSC800W', 'MUSC830W', 'MUSC830W']",NaN,professor,[],Aaron Goldman,goldman_aaron,Aaron Goldman,2013,"Goldman, Aaron",ARHU-School of Music,College of Arts & Humanities,Lecturer,"$13,072.32",Aaron Goldman
1,"['MUSC800W', 'MUSC830W', 'MUSC830W']",NaN,professor,[],Aaron Goldman,goldman_aaron,Aaron Goldman,2014,"Goldman, Aaron",ARHU-School of Music,College of Arts & Humanities,Lecturer,"$20,171.52",Aaron Goldman
2,"['MUSC800W', 'MUSC830W', 'MUSC830W']",NaN,professor,[],Aaron Goldman,goldman_aaron,Aaron Goldman,2015,"Goldman, Aaron",ARHU-School of Music,College of Arts & Humanities,Lecturer,"$17,180.08",Aaron Goldman
3,"['MUSC800W', 'MUSC830W', 'MUSC830W']",NaN,professor,[],Aaron Goldman,goldman_aaron,Aaron Goldman,2016,"Goldman, Aaron",ARHU-School of Music,College of Arts & Humanities,Lecturer,"$13,733.78",Aaron Goldman
4,"['MUSC800W', 'MUSC830W', 'MUSC830W']",NaN,professor,[],Aaron Goldman,goldman_aaron,Aaron Goldman,2017,"Goldman, Aaron",ARHU-School of Music,College of Arts & Humanities,Lecturer,"$21,809.45",Aaron Goldman
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30599,[],NaN,professor,[],Theresa Nebel Robinson,robinson,Theresa Nebel Robinson,2018,"Robinson, Theresa Nebel","EDUC-Teaching and Learning, Policy and Leadership",College of Education,Sr Fac Spec,"$70,000.00",Theresa Nebel Robinson
30600,[],NaN,professor,[],Theresa Nebel Robinson,robinson,Theresa Nebel Robinson,2019,"Robinson, Theresa Nebel","EDUC-Teaching and Learning, Policy and Leadership",College of Education,Sr Fac Spec,"$71,400.00",Theresa Nebel Robinson
30601,[],NaN,professor,[],Theresa Nebel Robinson,robinson,Theresa Nebel Robinson,2020,"Robinson, Theresa Nebel","EDUC-Teaching and Learning, Policy and Leadership",College of Education,Senior Faculty Specialist,"$74,648.81",Theresa Nebel Robinson
30602,[],NaN,professor,[],Theresa Nebel Robinson,robinson,Theresa Nebel Robinson,2021,"Robinson, Theresa Nebel","EDUC-Teaching and Learning, Policy and Leadership",College of Education,Senior Faculty Specialist,"$76,141.79",Theresa Nebel Robinson


In [6]:
merge_rmp = fuzzy_merge(rmp_df, salaries_df, fuzz_on="name", how="inner")
merge_rmp

,name_x,rating,courses,reviews,name,year,employee,department,division,title,salary,name_y
0,Eyad Abed,0.000,[],[],Eyad H Abed,2013,"Abed, Eyad H",ENGR-Electrical & Computer Engineering,A. James Clark School of Engineering,Prof,"$216,648.00",Eyad H Abed
1,Eyad Abed,0.000,[],[],Eyad H Abed,2014,"Abed, Eyad H",ENGR-Electrical & Computer Engineering,A. James Clark School of Engineering,Prof,"$226,494.65",Eyad H Abed
2,Eyad Abed,0.000,[],[],Eyad H Abed,2015,"Abed, Eyad H",ENGR-Electrical & Computer Engineering,A. James Clark School of Engineering,Prof,"$233,912.35",Eyad H Abed
3,Eyad Abed,0.000,[],[],Eyad H Abed,2016,"Abed, Eyad H",ENGR-Electrical & Computer Engineering,A. James Clark School of Engineering,Prof,"$233,912.35",Eyad H Abed
4,Eyad Abed,0.000,[],[],Eyad H Abed,2017,"Abed, Eyad H",ENGR-Electrical & Computer Engineering,A. James Clark School of Engineering,Prof,"$235,783.65",Eyad H Abed
...,...,...,...,...,...,...,...,...,...,...,...,...
85000,Lindsey Wareing,0.000,[],[],Lindsey M. Wareing,2022,"Wareing, Lindsey M.",VPUR-Univ Alumni Association,VP University\nRelations,Director,"$102,010.00",Lindsey M. Wareing
85001,Bradley Bower,3.625,"['TEC300', 'TEC366', 'TEC325']","[{'professor': 'Bradley Bower', 'course': 'TEC...",Bradley S. Bower,2022,"Bower, Bradley S.",VPUR-VP University Relations,VP University\nRelations,Asst Prog Dir,"$78,780.00",Bradley S. Bower
85002,Michelle Christian,3.200,"['SPCH125', 'JOUR201', 'COMM394']","[{'professor': 'Michelle Christian', 'course':...",Michelle E. Christian,2022,"Christian, Michelle E.",VPUR-VP University Relations,VP University\nRelations,Asst Prog Dir,"$82,567.50",Michelle E. Christian
85003,Sarah Tabaka,0.000,[],[],Sarah M. Tabaka,2022,"Tabaka, Sarah M.",VPUR-VP University Relations,VP University\nRelations,Coordinator,"$63,000.00",Sarah M. Tabaka


In [17]:
reviews = []

for name, rows in merge_pt.merge(merge_rmp, how="outer").groupby("name"):
    for rs in map(literal_eval, rows["reviews"].unique()):
        reviews += rs

reviews = pd.DataFrame(reviews)
reviews = reviews.drop(columns=["expected_grade"])
reviews["created"] = pd.to_datetime(reviews["created"].str.replace("UTC", ""))


In [18]:
reviews

,professor,course,review,rating,created
0,A Kruglanski,PSYC489H,"DO NOT TAKE PSYC489H ""Motivated Social Cogniti...",2,2015-09-07 18:44:00+00:00
1,A.U. Shankar,CMSC412,Lectures are pretty dry and difficult to follo...,3,2013-01-02 21:32:00+00:00
2,A.U. Shankar,CMSC412,"Professor: He does have a stutter, but if you ...",3,2012-12-23 03:51:00+00:00
3,A.U. Shankar,CMSC412,This is a horrible class. The projects are imp...,1,2012-10-29 00:54:00+00:00
4,A.U. Shankar,CMSC412,I have a lot of respect for Dr. Shankar. He is...,5,2012-05-24 13:00:00+00:00
...,...,...,...,...,...
93786,Zubin Jelveh,CCJS418E,Possibly the most clueless professor regarding...,1,2022-04-07 19:01:06+00:00
93787,chrysoula papagianni,BIO208L,Talks very slowly. VERY picky about wording on...,2,2019-08-12 02:34:09+00:00
93788,chrysoula papagianni,BIO209L,Nice teacher. Explains the lab very well. Quiz...,5,2014-01-12 05:48:35+00:00
93789,sahar jendi,ARAB100,She is a great professor,5,2016-05-22 23:32:24+00:00


In [19]:
reviews[reviews.professor == "Justin Wyss-Gallifent"]

,professor,course,review,rating,created
46827,Justin Wyss-Gallifent,MATH246,"Best Math professor I've ever had at UMD, hand...",5,2016-08-25 17:52:00+00:00
46828,Justin Wyss-Gallifent,MATH241,"He is a great teacher I highly recommend him, ...",5,2015-06-03 11:41:00+00:00
46829,Justin Wyss-Gallifent,MATH241,"Justin is one of the best, if not the best pro...",5,2015-02-02 03:24:00+00:00
46830,Justin Wyss-Gallifent,MATH206,"Didn't think he would live up to the hype, but...",5,2015-01-28 02:08:00+00:00
46831,Justin Wyss-Gallifent,MATH241,Justin is awesome! I had the pleasure of him b...,5,2015-01-12 00:00:00+00:00
...,...,...,...,...,...
47114,Justin Wyss-Gallifent,MATH241,Cool and nice guy from somewhere in Europe&#63...,5,2005-05-12 22:08:33+00:00
47115,Justin Wyss-Gallifent,MATH113,"Justin is an awesome teacher. He is nice, fai...",5,2005-05-06 09:58:24+00:00
47116,Justin Wyss-Gallifent,MATH140,Very energetic effective teacher. He makes sur...,5,2005-03-24 17:02:46+00:00
47117,Justin Wyss-Gallifent,MATH141,"Passionate about teaching, not burnt out like ...",5,2005-03-15 11:41:37+00:00
